In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("LIUM/tedlium", "release1", split = "train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.24k [00:00<?, ?B/s]

tedlium.py:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

The repository for LIUM/tedlium contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/LIUM/tedlium.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

test.tar.gz:   0%|          | 0.00/307M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
transcriptions = dataset['text']

In [ ]:
for i in range(10):
  print(transcriptions[i])

<sil> {COUGH} with {NOISE} sarah connor set {NOISE} the {UH}
the
<sil> {COUGH} that(2) {COUGH}
to {NOISE} <sil>
{UH} that {NOISE}
<sil> the <sil>
which(2) they entered(2) into a photo array {NOISE} days later and he was positively identified(2) <sil> now {UM} i'm going to leave you <sil> with a self {UH} portrait <sil>
and(2) {NOISE} it {NOISE} reiterates {SMACK} that {UH} distortion {NOISE} is a constant <sil> and our eyes are {COUGH} easily deceived {SMACK} <sil>
<sil> you {COUGH}
i wanted to confront the boundaries(2) of the citizen(2) <sil> self imposed and real {SMACK} and {NOISE} confront the divide {COUGH} between privileged(4) {SMACK} and public {NOISE} access <sil> to knowledge(2) <sil> <sil>


In [ ]:
import csv

In [ ]:
with open("output.csv", "w", newline = "") as file:
    writer = csv.writer(file)
    for transcription in transcriptions:
        writer.writerow([transcription])

NameError: name 'transcriptions' is not defined

In [ ]:
print(len(transcriptions))

56803


In [ ]:
data = []
with open("output.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

In [ ]:
print(len(data))

56803


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# compute tf-idf
vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = vectorizer.fit_transform(data)

# sum score for each argument
tfidf_array = tfidf_matrix.toarray()
counts = np.sum(tfidf_array > 0, axis = 0)
scores = np.sum(tfidf_array, axis = 1) / counts

# normalize between 0 and 1
scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(scores.reshape(-1, 1))

# classify score
def classify_score(score):
    if score <= 0.33:
        return "weak"
    elif score <= 0.66:
        return "average"
    else:
        return "strong"

# display results
for i, argument in enumerate(data):
    print(f"Argument: {argument}")
    print(f"Score: {normalized_scores[i][0]}")
    print(f"Classification: {classify_score(normalized_scores[i][0])}")
    print()